# Azure AI Search met NVIDIA NIM en LlamaIndex-integratie

In dit notebook laten we zien hoe je NVIDIA's AI-modellen en LlamaIndex kunt gebruiken om een krachtige Retrieval-Augmented Generation (RAG)-pipeline te creëren. We maken gebruik van NVIDIA's LLMs en embeddings, integreren deze met Azure AI Search als vector store, en voeren RAG uit om de zoekkwaliteit en efficiëntie te verbeteren.

## Voordelen
- **Schaalbaarheid**: Maak gebruik van NVIDIA's grote taalmodellen en Azure AI Search voor schaalbare en efficiënte zoekopdrachten.
- **Kostenbesparing**: Optimaliseer zoeken en ophalen met efficiënte vectoropslag en hybride zoektechnieken.
- **Hoge prestaties**: Combineer krachtige LLMs met gevectoriseerde zoekopdrachten voor snellere en nauwkeurigere antwoorden.
- **Kwaliteit**: Behoud hoge zoekkwaliteit door LLM-antwoorden te onderbouwen met relevante opgehaalde documenten.

## Vereisten
- 🐍 Python 3.9 of hoger
- 🔗 [Azure AI Search Service](https://learn.microsoft.com/azure/search/)
- 🔗 NVIDIA API-sleutel voor toegang tot NVIDIA's LLMs en Embeddings via de NVIDIA NIM-microservices

## Behandelde functies
- ✅ NVIDIA LLM-integratie (we gebruiken [Phi-3.5-MOE](https://build.nvidia.com/microsoft/phi-3_5-moe))
- ✅ NVIDIA Embeddings (we gebruiken [nv-embedqa-e5-v5](https://build.nvidia.com/nvidia/nv-embedqa-e5-v5))
- ✅ Geavanceerde ophaalmodi van Azure AI Search
- ✅ Documentindexering met LlamaIndex
- ✅ RAG met Azure AI Search en LlamaIndex in combinatie met NVIDIA LLMs

Laten we beginnen!


In [ ]:
!pip install azure-search-documents==11.5.1
!pip install --upgrade llama-index
!pip install --upgrade llama-index-core
!pip install --upgrade llama-index-readers-file
!pip install --upgrade llama-index-llms-nvidia
!pip install --upgrade llama-index-embeddings-nvidia
!pip install --upgrade llama-index-postprocessor-nvidia-rerank
!pip install --upgrade llama-index-vector-stores-azureaisearch
!pip install python-dotenv

## Installatie en Vereisten
Maak een Python-omgeving aan met Python versie >3.10.

## Aan de slag!


Om te beginnen heb je een `NVIDIA_API_KEY` nodig om NVIDIA AI Foundation-modellen te gebruiken:
1) Maak een gratis account aan bij [NVIDIA](https://build.nvidia.com/explore/discover).
2) Klik op het model van jouw keuze.
3) Onder Input, selecteer het Python-tabblad en klik op **Get API Key** en vervolgens op **Generate Key**.
4) Kopieer en bewaar de gegenereerde sleutel als NVIDIA_API_KEY. Vanaf daar heb je toegang tot de endpoints.


In [3]:
import getpass
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key


## RAG Voorbeeld met LLM en Embedding
### 1) Initialiseer de LLM
`llama-index-llms-nvidia`, ook bekend als NVIDIA's LLM-connector, stelt je in staat om verbinding te maken met en te genereren vanuit compatibele modellen die beschikbaar zijn in de NVIDIA API-catalogus. Zie hier voor een lijst van chat completion modellen: https://build.nvidia.com/search?term=Text-to-Text

Hier zullen we **mixtral-8x7b-instruct-v0.1** gebruiken.


In [75]:
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA

# Here we are using mixtral-8x7b-instruct-v0.1 model from API Catalog
Settings.llm = NVIDIA(model="microsoft/phi-3.5-moe-instruct", api_key=os.getenv("NVIDIA_API_KEY"))

### 2) Initialiseer de Embedding
`llama-index-embeddings-nvidia`, ook bekend als NVIDIA's Embeddings connector, stelt je in staat om verbinding te maken met en te genereren vanuit compatibele modellen die beschikbaar zijn in de NVIDIA API-catalogus. We hebben `nvidia/nv-embedqa-e5-v5` geselecteerd als het embedding model. Zie hier voor een lijst van tekst embedding modellen: https://build.nvidia.com/nim?filters=usecase%3Ausecase_text_to_embedding%2Cusecase%3Ausecase_image_to_embedding


In [6]:
from llama_index.embeddings.nvidia import NVIDIAEmbedding

Settings.embed_model = NVIDIAEmbedding(model="nvidia/nv-embedqa-e5-v5", api_key=os.getenv("NVIDIA_API_KEY"))

### 3) Een Azure AI Search Vector Store maken


In [76]:
import logging
import sys
import os
import getpass
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from IPython.display import Markdown, display
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement


search_service_api_key = os.getenv('AZURE_SEARCH_ADMIN_KEY') or getpass.getpass('Enter your Azure Search API key: ')
search_service_endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT') or getpass.getpass('Enter your Azure Search service endpoint: ')
search_service_api_version = "2024-07-01"
credential = AzureKeyCredential(search_service_api_key)

# Index name to use
index_name = "llamaindex-nvidia-azureaisearch-demo"

# Use index client to demonstrate creating an index
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=credential,
)

# Use search client to demonstrate using existing index
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=credential,
)

In [ ]:
vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1024, # dimensionality for nv-embedqa-e5-v5 model
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
    # compression_type="binary" # Option to use "scalar" or "binary". NOTE: compression is only supported for HNSW
)

In [20]:
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.text_splitter import TokenTextSplitter

# Configure text splitter (nv-embedqa-e5-v5 model has a limit of 512 tokens per input size)
text_splitter = TokenTextSplitter(separator=" ", chunk_size=500, chunk_overlap=10)

# Load documents
documents = SimpleDirectoryReader(
    input_files=["data/txt/state_of_the_union.txt"]
).load_data()
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index with text splitter
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
    storage_context=storage_context,
)

### 5) Maak een Query Engine om vragen te stellen over je data

Hier is een query die gebruikmaakt van pure vectorzoekopdrachten in Azure AI Search en de respons baseert op onze LLM (Phi-3.5-MOE)


In [69]:
query_engine = index.as_query_engine()
response = query_engine.query("Who did the speaker mention as being present in the chamber?")
display(Markdown(f"{response}"))

 The speaker mentioned the Ukrainian Ambassador to the United States, along with other members of Congress, the Cabinet, and various officials such as the Vice President, the First Lady, and the Second Gentleman, as being present in the chamber.

Hier is een query met hybride zoeken in Azure AI Search.


In [70]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from IPython.display import Markdown, display
from llama_index.core.schema import MetadataMode

# Initialize hybrid retriever and query engine
hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.HYBRID)
hybrid_query_engine = RetrieverQueryEngine(retriever=hybrid_retriever)

# Query execution
query = "What were the exact economic consequences mentioned in relation to Russia's stock market?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))

 The Russian stock market experienced a significant drop, losing 40% of its value. Additionally, trading had to be suspended due to the ongoing situation.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin. 

I spent countless hours unifying our European allies. We shared with the world in advance what we knew Putin was planning and precisely how he would try to falsely justify his aggression.  

We countered Russia’s lies with truth.   

And now that he has acted the free world is holding him accountable. 

Along with twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland. 

We are inflicting pain on Russia and supporting the people of Ukraine. Putin is now isolated from the world more than ever. 

Together with our allies –we are right now enforcing powerful economic sanctions. 

We are cutting off Russia’s largest banks from the international financial system.  



#### Analyse van Vectorzoekopdrachten
De reactie van het LLM geeft nauwkeurig de belangrijkste economische gevolgen weer die in de brontekst worden genoemd met betrekking tot de Russische aandelenmarkt. Het stelt specifiek dat de Russische aandelenmarkt een aanzienlijke daling heeft doorgemaakt, waarbij 40% van de waarde verloren ging, en dat de handel werd opgeschort vanwege de huidige situatie. Deze reactie sluit goed aan bij de informatie uit de bron, wat aangeeft dat het LLM de relevante details over de impact op de aandelenmarkt als gevolg van de acties van Rusland en de opgelegde sancties correct heeft geïdentificeerd en samengevat.

#### Commentaar op Bronknopen
De bronknopen geven een gedetailleerd verslag van de economische gevolgen die Rusland ondervond door internationale sancties. De tekst benadrukt dat de Russische aandelenmarkt 40% van zijn waarde verloor en dat de handel werd opgeschort. Daarnaast worden andere economische gevolgen genoemd, zoals de waardevermindering van de Roebel en de bredere isolatie van de Russische economie. De reactie van het LLM heeft de belangrijkste punten uit deze knopen effectief samengevat, met de nadruk op de impact op de aandelenmarkt zoals gevraagd in de query.


Laten we eens kijken naar een vraag waarbij Hybrid Search geen goed onderbouwd antwoord geeft:


In [71]:
# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it does mention that the events discussed are happening in the current era and that the actions taken are in response to Putin's aggression. For the precise date, one would need to refer to external sources or historical records.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  

Our forces are not going to Europe to fight in Ukraine, but to defend our NATO Allies – in the event that Putin decides to keep moving west.  

For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia. 

As I have made crystal clear the United States and our Allies will defend every inch of territory of NATO countries with the full force of our collective power.  

And we remain clear-eyed. The Ukrainians are fighting back with pure courage. But the next few days weeks, months, will be hard on them.  

Putin has unleashed violence and chaos.  But while he may make gains on the battlefield – he will pay a continuing high price over the long run. 

And a proud Ukrainian people, who have known 30 years  of independence,

### Hybride Zoekopdracht: Analyse van LLM-reacties
De LLM-reactie in het voorbeeld van de Hybride Zoekopdracht geeft aan dat de verstrekte context niet de exacte datum van de invasie van Oekraïne door Rusland specificeert. Deze reactie suggereert dat de LLM gebruikmaakt van de beschikbare informatie in de brondocumenten, maar erkent het ontbreken van precieze details in de tekst.

De reactie is nauwkeurig in het identificeren dat de context gebeurtenissen vermeldt die verband houden met de agressie van Rusland, maar niet de specifieke invasiedatum aangeeft. Dit toont aan dat de LLM in staat is om de verstrekte informatie te begrijpen, terwijl het ook hiaten in de inhoud herkent. De LLM moedigt de gebruiker effectief aan om externe bronnen of historische gegevens te raadplegen voor de exacte datum, wat een mate van voorzichtigheid laat zien wanneer informatie onvolledig is.

### Analyse van Bronnodes
De bronnodes in het voorbeeld van de Hybride Zoekopdracht bevatten fragmenten uit een toespraak die de reactie van de VS op de acties van Rusland in Oekraïne bespreekt. Deze nodes benadrukken de bredere geopolitieke impact en de stappen die de VS en haar bondgenoten hebben genomen in reactie op de invasie, maar ze vermelden niet de specifieke invasiedatum. Dit komt overeen met de LLM-reactie, die correct identificeert dat de context de precieze datuminformatie mist.


In [72]:
# Initialize hybrid retriever and query engine
semantic_reranker_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID)
semantic_reranker_query_engine = RetrieverQueryEngine(retriever=semantic_reranker_retriever)

# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = semantic_reranker_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it mentions that the event occurred six days before the speech was given. To determine the precise date, one would need to know the date of the speech.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

### Hybride met herordening: Analyse van LLM-antwoorden
In het voorbeeld van Hybride met herordening biedt het LLM-antwoord extra context door te vermelden dat het evenement zes dagen vóór de toespraak plaatsvond. Dit geeft aan dat het LLM in staat is om de datum van de invasie af te leiden op basis van de timing van de toespraak, hoewel het nog steeds de exacte datum van de toespraak moet kennen voor precisie.

Dit antwoord toont een verbeterd vermogen om contextuele aanwijzingen te gebruiken om een meer informatief antwoord te geven. Het benadrukt het voordeel van herordening, waarbij het LLM toegang heeft tot en prioriteit geeft aan relevantere informatie om een nauwkeuriger benadering van het gewenste detail te bieden (d.w.z. de datum van de invasie).

### Analyse van bronknooppunten
De bronknooppunten in dit voorbeeld bevatten verwijzingen naar de timing van de invasie van Rusland, waarbij specifiek wordt vermeld dat deze zes dagen vóór de toespraak plaatsvond. Hoewel de exacte datum nog steeds niet expliciet wordt genoemd, bieden de knooppunten temporele context die het LLM in staat stelt een meer genuanceerd antwoord te geven. Het opnemen van dit detail laat zien hoe herordening de mogelijkheid van het LLM kan verbeteren om informatie uit de gegeven context te extraheren en af te leiden, wat resulteert in een nauwkeuriger en informatiever antwoord.


**Opmerking:**
In dit notebook hebben we gebruik gemaakt van NVIDIA NIM-microservices uit de NVIDIA API Catalog.  
De bovenstaande API's, `NVIDIA (llms)`, `NVIDIAEmbedding`, en [Azure AI Search Semantic Hybrid Retrieval (built-in reranking)](https://learn.microsoft.com/azure/search/semantic-search-overview). Let op, de bovenstaande API's kunnen ook zelf-gehoste microservices ondersteunen.

**Voorbeeld:**
```python
NVIDIA(model="meta/llama3-8b-instruct", base_url="http://your-nim-host-address:8000/v1")```



---

**Disclaimer**:  
Dit document is vertaald met behulp van de AI-vertalingsservice [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we streven naar nauwkeurigheid, dient u zich ervan bewust te zijn dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in zijn oorspronkelijke taal moet worden beschouwd als de gezaghebbende bron. Voor cruciale informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor eventuele misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
